In [ ]:
# download the metadata for all images, 50000 train images, 50000 test images
from huggingface_hub import snapshot_download
snapshot_download(repo_id="osv5m/osv5m", local_dir="datasets/osv5m", allow_patterns=['images/train/00.zip','images/test/00.zip','*.csv'], repo_type='dataset')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

test.csv:   0%|          | 0.00/116M [00:00<?, ?B/s]

00.zip:   0%|          | 0.00/2.52G [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/2.92G [00:00<?, ?B/s]

00.zip:   0%|          | 0.00/2.25G [00:00<?, ?B/s]

In [2]:
# extract zips
import os
import zipfile
for root, dirs, files in os.walk("datasets/osv5m"):
    for file in files:
        if file.endswith(".zip"):
            with zipfile.ZipFile(os.path.join(root, file), 'r') as zip_ref:
                zip_ref.extractall(root)
                os.remove(os.path.join(root, file))


In [4]:
import torch
import numpy as np
import pandas as pd
import csv
# Haversine distance - length of an arc between two points on a sphere
# https://github.com/gastruc/osv5m/blob/main/metrics/utils.py
def haversine(pred, gt):
    # expects inputs to be np arrays in (lat, lon) format as radians
    # N x 2

    # calculate the difference in latitude and longitude between the predicted and ground truth points
    lat_diff = pred[:, 0] - gt[:, 0]
    lon_diff = pred[:, 1] - gt[:, 1]

    # calculate the haversine formula components
    lhs = torch.sin(lat_diff / 2) ** 2
    rhs = torch.cos(pred[:, 0]) * torch.cos(gt[:, 0]) * torch.sin(lon_diff / 2) ** 2
    a = lhs + rhs

    # calculate the final distance using the haversine formula
    c = 2 * torch.arctan2(torch.sqrt(a), torch.sqrt(1 - a))
    distance = 6371 * c

    return distance
davis = np.array([[43.547504,-79.662211]])
glasgow = np.array([[55.853467, -4.249420]])

davis_tensor = torch.tensor(davis, dtype=torch.float32).deg2rad()
glasgow_tensor = torch.tensor(glasgow, dtype=torch.float32).deg2rad()

haversine(davis_tensor, glasgow_tensor)

tensor([5307.1450])

In [15]:
def predict_utm(image_id) -> list[float]:
  # returns the predicted latitude and longitude of a given image
  return [43.547504,-79.662211]

In [33]:
# https://github.com/gastruc/osv5m/blob/main/metrics/distance_based.py
def geoscore(distance):
  return 5000 * torch.exp(-distance / 1492.7)

In [16]:
file = open('datasets/osv5m/test.csv', mode='r', newline='', encoding='utf-8')
reader = csv.reader(file)

test_gt = {}
for row in reader:
  if row[0] == 'id':
    continue
  test_gt[int(row[0])] = np.array([float(row[1]), float(row[2])])

file.close()
print(test_gt[1000055667540054])

[60.77272916 -0.83074192]


In [38]:
# evaluate a predictor by its average geoscore on the first n images from the test set

def evaluate(predictor, n):
  test_path = '/content/datasets/osv5m/images/test/00'
  i = 0
  geoscore_value = 0
  for image_id in os.listdir(test_path):
    id = int(image_id[:image_id.index('.')])
    prediction = np.array(predictor(id))
    gt = test_gt[id]

    prediction_torch = torch.tensor(prediction.reshape(1,2), dtype=torch.float32).deg2rad()
    test_gt_torch = torch.tensor(gt.reshape(1,2), dtype=torch.float32).deg2rad()

    distance = haversine(prediction_torch, test_gt_torch)
    geoscore_value += geoscore(distance)
    i += 1
    if i == n:
      break
  return geoscore_value / n

print("avg geoscore: ",evaluate(predict_utm, 10000).item())

avg geoscore:  307.60491943359375
